#### 고속버스

In [ ]:
import requests
import numpy as np
import pandas as pd
import statsmodels as ss
from bs4 import BeautifulSoup
import pprint
apikey = "0hCmBr0rnl14%2Fsz%2BuaqqAisxHYVdGOfyRJjD2AYXvUpLrTSxKw33PxHiKRU3TkNB9EZqhgSWT88XByl4D%2BiIPQ%3D%3D"
dttype = "json"

[도시코드 목록 만들기]

In [ ]:
url2_2 = "http://apis.data.go.kr/1613000/ExpBusInfoService/getCtyCodeList?serviceKey={}&_type={}".format(apikey, dttype)
expbus_bct = requests.get(url2_2).json()
type(expbus_bct)
expbus_bct

In [ ]:
expbus_bct_list = expbus_bct["response"]["body"]["items"]["item"]
expbus_bct_list

expbus_bct_dic = {}
expbus_bct_dic["exbs_bctID"] = list(m["cityCode"] for m in expbus_bct_list)
expbus_bct_dic["exbs_bctNM"] = list(m["cityName"] for m in expbus_bct_list)
expbus_bct_dic

expbus_bct_df = pd.DataFrame(expbus_bct_dic)
expbus_bct_df # 도시코드 DF 완료

[고속버스터미널 목록조회]

In [ ]:
url2_3 = "http://apis.data.go.kr/1613000/ExpBusInfoService/getExpBusTrminlList?serviceKey={}&_type={}&numOfRows=1000".format(apikey, dttype)

expbus_stn = requests.get(url2_3).json()
type(expbus_stn)
expbus_stn

In [ ]:
expbus_stn_list = expbus_stn["response"]["body"]["items"]["item"]
expbus_stn_list

expbus_stn_dic = {}
expbus_stn_dic["exbs_stnID"] = list(m["terminalId"] for m in expbus_stn_list)
expbus_stn_dic["exbs_stnNM"] = list(m["terminalNm"] for m in expbus_stn_list)
expbus_stn_dic

expbus_stn_df = pd.DataFrame(expbus_stn_dic)
expbus_stn_df # 고속버스터미널 목록 DF 완료

[출도착지 기반 고속버스정보 조회]: 출발지 ID/도착지 ID를 바꿔가며 요청-데이터 받아오기-리스트로 생성-딕셔너리에 반복 저장하는 과정 필요

In [ ]:
url2_4 = "http://apis.data.go.kr/1613000/ExpBusInfoService/getStrtpntAlocFndExpbusInfo?serviceKey={}&depTerminalId=NAEK010&arrTerminalId=NAEK300&depPlandTime=20230427&_type={}&numOfRows=1000".format(apikey, dttype)
expbus_rut = requests.get(url2_4).json()
type(expbus_rut)
expbus_rut
expbus_rut_list = expbus_rut["response"]["body"]["items"]["item"]
expbus_rut_list

In [ ]:
expbus_rut_list = expbus_rut["response"]["body"]["items"]["item"]
expbus_rut_list

expbus_rut_dic = {}
expbus_rut_dic["exbs_stnNM_dep"] = list(m["depPlaceNm"] for m in expbus_rut_list)
expbus_rut_dic["exbs_stnNM_arr"] = list(m["arrPlaceNm"] for m in expbus_rut_list)
expbus_rut_dic["exbs_schd"] = list(m["depPlandTime"] for m in expbus_rut_list)
expbus_rut_dic["exbs_grdNM"] = list(m["gradeNm"] for m in expbus_rut_list)
expbus_rut_dic["exbs_rutID"] = list(m["routeId"] for m in expbus_rut_list)

expbus_rut_dic

expbus_rut_df = pd.DataFrame(expbus_rut_dic)
expbus_rut_df.head(3)

In [ ]:
expbus_stncode = expbus_stn_df["exbs_stnID"]
#depday = ["20230427", "20230429", "20230430"]
depday = ["20230427"]

expbus_rut_dic = {}

for dep in expbus_stncode:
    for arr in expbus_stncode:
        if dep == arr:
            continue
        else:
            url2_4 = "http://apis.data.go.kr/1613000/expbusInfoService/getStrtpntAlocFndexpbusInfo?serviceKey={}&depPlaceId={}&arrPlaceId={}&depPlandTime={}&_type={}&numOfRows=1000".format(apikey, dep, arr, depday, dttype)
            expbus_rut = requests.get(url2_4).json()
            expbus_rut_list = expbus_rut["response"]["body"]["items"]["item"]

- Expecting value: line 1 column 1 (char 0) 왜 자꾸 오류
- 노선이 없는 경우는 어떻게